In [1]:
#import the necessary libs

from pyspark.sql import SparkSession
import getpass

username=getpass.getuser()
spark=SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.suffle.useOldFetchProtocol",'true'). \
    config("spark.dql.warehouse.dir",f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [3]:
loans_repay_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.load("/user/itv014478/lendingclubproject/raw/loan_repayment_data_csv")

In [4]:
loans_repay_raw_df

loan_id,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
68407277,821.72,0.0,4421.723916800001,122.67,Jan-2019,null
68355089,979.66,0.0,25679.66,926.35,Jun-2016,null
68341763,2705.92,0.0,22705.924293878397,15813.3,Jun-2017,null
66310712,12361.66,0.0,31464.01,829.9,Feb-2019,Apr-2019
68476807,1340.5,0.0,11740.5,10128.96,Jul-2016,null
68426831,1758.95,0.0,13708.9485297572,7653.56,May-2017,null
68476668,1393.8,0.0,21393.800000011,15681.05,Nov-2016,null
67275481,1538.51,0.0,21538.508976797,14618.23,Jan-2017,null
68466926,998.97,0.0,10998.9715749644,1814.48,Aug-2018,null
68616873,939.58,0.0,8939.5805031401,4996.24,Apr-2017,null


In [5]:
loans_repay_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_rec_int: double (nullable = true)
 |-- total_rec_late_fee: double (nullable = true)
 |-- total_pymnt: double (nullable = true)
 |-- last_pymnt_amnt: string (nullable = true)
 |-- last_pymnt_d: string (nullable = true)
 |-- next_pymnt_d: string (nullable = true)



In [6]:
loans_repay_schema = 'loan_id string, total_principal_received float, total_interest_received float, total_late_fee_received float, total_payment_received float, last_payment_amount float, last_payment_date string, next_payment_date string'

In [8]:
loans_repay_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loans_repay_schema) \
.load("/user/itv014478/lendingclubproject/raw/loan_repayment_data_csv")

In [9]:
loans_repay_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)



In [10]:
from pyspark.sql.functions import current_timestamp

In [11]:
loans_repay_df_ingestd = loans_repay_raw_df.withColumn("ingest_date", current_timestamp())

In [12]:
loans_repay_df_ingestd

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
68407277,821.72,0.0,4421.724,122.67,null,null,null,2024-11-19 03:34:...
68355089,979.66,0.0,25679.66,926.35,null,null,null,2024-11-19 03:34:...
68341763,2705.92,0.0,22705.924,15813.3,null,null,null,2024-11-19 03:34:...
66310712,12361.66,0.0,31464.01,829.9,null,Apr-2019,null,2024-11-19 03:34:...
68476807,1340.5,0.0,11740.5,10128.96,null,null,null,2024-11-19 03:34:...
68426831,1758.95,0.0,13708.948,7653.56,null,null,null,2024-11-19 03:34:...
68476668,1393.8,0.0,21393.8,15681.05,null,null,null,2024-11-19 03:34:...
67275481,1538.51,0.0,21538.51,14618.23,null,null,null,2024-11-19 03:34:...
68466926,998.97,0.0,10998.972,1814.48,null,null,null,2024-11-19 03:34:...
68616873,939.58,0.0,8939.58,4996.24,null,null,null,2024-11-19 03:34:...


In [13]:
loans_repay_df_ingestd.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [14]:
loans_repay_df_ingestd.count()

462494

In [15]:
loans_repay_df_ingestd.createOrReplaceTempView("loan_repayments")

In [16]:
spark.sql("select count(*) from loan_repayments where total_principal_received is null")

count(1)
2


In [20]:
columns_to_check = ["total_principal_received", "total_interest_received", "total_late_fee_received", "total_payment_received"]

In [21]:
loans_repay_filtered_df = loans_repay_df_ingestd.na.drop(subset=columns_to_check)

In [22]:
loans_repay_filtered_df.count()

462491

In [23]:
loans_repay_filtered_df.createOrReplaceTempView("loan_repayments")

In [24]:
spark.sql("select count(*) from loan_repayments where total_payment_received = 0.0")

count(1)
396


In [25]:
spark.sql("select count(*) from loan_repayments where total_payment_received = 0.0 and total_principal_received != 0.0")

Py4JJavaError: An error occurred while calling o161.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: ResultStage 21 (showString at NativeMethodAccessorImpl.java:0) has failed the maximum allowable number of times: 4. Most recent failure reason: org.apache.spark.shuffle.FetchFailedException: java.lang.IllegalArgumentException: Unknown message type: 9 	at org.apache.spark.network.shuffle.protocol.BlockTransferMessage$Decoder.fromByteBuffer(BlockTransferMessage.java:71) 	at org.apache.spark.network.shuffle.ExternalShuffleBlockHandler.receive(ExternalShuffleBlockHandler.java:80) 	at org.apache.spark.network.server.TransportRequestHandler.processRpcRequest(TransportRequestHandler.java:180) 	at org.apache.spark.network.server.TransportRequestHandler.handle(TransportRequestHandler.java:103) 	at org.apache.spark.network.server.TransportChannelHandler.channelRead(TransportChannelHandler.java:118) 	at org.spark_project.io.netty.channel.AbstractChannelHandlerContex

count(1)
37


In [26]:
spark.sql("select * from loan_repayments where total_payment_received = 0.0 and total_principal_received != 0.0")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
68373395,841.01,0.0,10841.013,0.0,null,null,null,2024-11-19 03:38:...
67999698,2735.37,0.0,19535.371,0.0,null,null,null,2024-11-19 03:38:...
65945334,1206.41,0.0,12206.413,0.0,null,null,null,2024-11-19 03:38:...
66064197,6527.91,0.0,29527.914,0.0,null,null,null,2024-11-19 03:38:...
65874763,5441.86,0.0,25441.861,0.0,null,null,null,2024-11-19 03:38:...
65601746,2376.69,0.0,14451.691,0.0,null,null,null,2024-11-19 03:38:...
63444672,5041.51,0.0,25041.512,0.0,null,null,null,2024-11-19 03:38:...
61504248,1334.93,0.0,14834.932,0.0,null,null,null,2024-11-19 03:38:...
61473016,1137.38,0.0,17137.383,0.0,null,null,null,2024-11-19 03:38:...
61361777,1093.83,0.0,11893.827,0.0,null,null,null,2024-11-19 03:38:...


In [28]:
from pyspark.sql.functions import when, col
loans_payments_fixed_df = loans_repay_filtered_df.withColumn(
   "total_payment_received",
    when(
        (col("total_principal_received") != 0.0) &
        (col("total_payment_received") == 0.0),
        col("total_principal_received") + col("total_interest_received") + col("total_late_fee_received")
    ).otherwise(col("total_payment_received"))
)

In [29]:
loans_payments_fixed_df

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
68407277,821.72,0.0,4421.724,122.67,null,null,null,2024-11-19 03:41:...
68355089,979.66,0.0,25679.66,926.35,null,null,null,2024-11-19 03:41:...
68341763,2705.92,0.0,22705.924,15813.3,null,null,null,2024-11-19 03:41:...
66310712,12361.66,0.0,31464.01,829.9,null,Apr-2019,null,2024-11-19 03:41:...
68476807,1340.5,0.0,11740.5,10128.96,null,null,null,2024-11-19 03:41:...
68426831,1758.95,0.0,13708.948,7653.56,null,null,null,2024-11-19 03:41:...
68476668,1393.8,0.0,21393.8,15681.05,null,null,null,2024-11-19 03:41:...
67275481,1538.51,0.0,21538.51,14618.23,null,null,null,2024-11-19 03:41:...
68466926,998.97,0.0,10998.972,1814.48,null,null,null,2024-11-19 03:41:...
68616873,939.58,0.0,8939.58,4996.24,null,null,null,2024-11-19 03:41:...


In [31]:
loans_payments_fixed_df.filter("loan_id == '59601218'")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
59601218,625.0,15.0,3640.0002,4280.0,null,null,null,2024-11-19 03:43:...


In [32]:
loans_payments_fixed2_df = loans_payments_fixed_df.filter("total_payment_received != 0.0")

In [35]:
loans_payments_fixed2_df.filter("last_payment_date is null").count()

382863

In [37]:
loans_payments_fixed2_df.write \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv014478/lendingclubproject/cleaned/loans_repayments_parquet") \
.save()